<a href="https://colab.research.google.com/github/deagwon97/algorithm/blob/master/notebook/train/%5BHK%5Dremake_sam_unet_efb7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import clear_output
!pip install segmentation_models_pytorch
!pip install albumentations==0.3.2
!pip install -U git+https://github.com/albu/albumentations --no-cache-dir # albu.lambda 지원을 위해 설치
!pip install wandb
clear_output()

     |████████████████████████████████| 71kB 3.5MB/s 
     |████████████████████████████████| 245kB 7.7MB/s 
     |████████████████████████████████| 61kB 8.2MB/s 
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-cp36-none-any.whl size=12421 sha256=49b4df94dbe348c178f071a7e276a6312a46b18c19ad85bf30b6e6c3f328355a
  Stored in directory: /root/.cache/pip/wheels/42/1e/a9/2a578ba9ad04e776e80bf0f70d8a7f4c29ec0718b92d8f6ccd
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-cp36-none-any.whl size=60963 sha256=c85cbcda213c2b75c2df2932f7f3995ce8419edfb117489a0bce555815262d5e
  Stored in directory: /root/.cache/pip/wheels/69/df/63/62583c096289713f22db605aa2334de5b591d59861a02c2ecd
Successfully built efficientnet-pytorch pretrainedmodels
     |████████████████████████████████| 81kB 3.8MB/s 
     |████████████████████████████████| 37.6MB 145kB/s 
     |████████████████████████████████| 634kB 71.1MB/s 
  Created wheel for albumentations: filename=album

In [ ]:
from PIL import Image
import tifffile as tiff
import subprocess
import pandas as pd
from IPython.display import clear_output
import matplotlib.pyplot as plt
import sys
import numpy as np
import cv2
import os
from tqdm import tqdm as tqdm
import random
import zipfile
from sklearn.model_selection import train_test_split
from collections import defaultdict
import wandb
import math
import torchvision
import torchvision.transforms as transforms
import torch
from torch.utils.data import TensorDataset, DataLoader,Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.optimizer import Optimizer
from torch.optim import lr_scheduler
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR

import albumentations as albu
#from albumentations import torch as AT
from albumentations import Compose,Resize,OneOf,RandomBrightness,RandomContrast,Normalize,HorizontalFlip,Blur,ElasticTransform,GridDistortion,OpticalDistortion,GaussNoise 
from albumentations.pytorch import ToTensor
import segmentation_models_pytorch as smp
train_meta = pd.read_csv('/content/drive/MyDrive/kaggledrive/kidney/new_data/train.csv').set_index('id')

seed = 1015
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if not os.path.exists("train_object_512"):
    !mkdir /content/train_object_512
    !mkdir /content/masks_object_512

    path = "/content/drive/MyDrive/kaggledrive/kidney/remake_data2/"
    !cp "{path}train_object_512.zip" "/content/train_object_512/train_object.zip"
    !cp "{path}masks_object_512.zip" "/content/masks_object_512/masks_object.zip "

    !unzip "/content/train_object_512/train_object.zip"  -d "/content/train_object_512"
    !unzip "/content/masks_object_512/masks_object.zip " -d "/content/masks_object_512"

    clear_output()

In [ ]:
!cp "/content/drive/MyDrive/kaggledrive/kidney/adaboosting/fp_img.zip"  "fp_img.zip"
!cp "/content/drive/MyDrive/kaggledrive/kidney/adaboosting/fp_mask.zip" "fp_mask.zip"

!cp "/content/drive/MyDrive/kaggledrive/kidney/adaboosting/fn_img.zip"  "fn_img.zip"
!cp "/content/drive/MyDrive/kaggledrive/kidney/adaboosting/fn_mask.zip" "fn_mask.zip"

if not os.path.exists("fp_img"):
    !mkdir /content/fp_img
    !mkdir /content/fp_mask
    !mkdir /content/fn_img
    !mkdir /content/fn_mask   

    !unzip "fp_img.zip"   -d "/content/fp_img" 
    !unzip "fp_mask.zip"  -d "/content/fp_mask" 
    !unzip "fn_img.zip"   -d "/content/fn_img" 
    !unzip "fn_mask.zip"  -d "/content/fn_mask" 
    clear_output()

In [ ]:
fp_img = []
with zipfile.ZipFile("/content/fp_img.zip", 'r') as img_arch:
    fp_img_list = img_arch.namelist().copy()
fp_list = list(map(lambda x: str(x)[4:],fp_img_list))


fn_img = []
with zipfile.ZipFile("/content/fn_img.zip", 'r') as img_arch:
    fn_img_list = img_arch.namelist().copy()
fn_list = list(map(lambda x: str(x)[4:],fn_img_list))

print(len(fn_list))
print(len(fp_img_list))

In [ ]:
print(len(fp_img_list) / len(namelist))
print(len(fn_img_list) / len(namelist))

In [ ]:
class Kidney_Dataset(torch.utils.data.Dataset):
    def __init__(self, namelist, 
                 transform=None,
                 preprocessing=None,
                 classes=1, 
                 augmentation=None, 
                ):
        self.namelist = namelist
        self.transforms = transform
        self.classes = classes
        self.preprocessing = preprocessing
        self.augmentation = augmentation
        self.imgsize = 512

        self.to_tensor = Compose([
                                  albu.Lambda(image= to_tensor, mask=to_tensor),
                               ])
        
        
    def __getitem__(self,index):
        img_name = self.namelist[index]
        # Read Data----------------------------------------
        if np.random.rand(1)[0] < 0.15:
        #if np.random.rand(1)[0] > 0:
            if np.random.rand(1)[0] < 0.6:
                fp_idx = np.random.randint(len(fp_list))
                img = np.array(Image.open(f"fp_img/img_{fp_list[fp_idx]}"))
                mask = np.array(Image.open(f"fp_mask/mask_{fp_list[fp_idx]}"))[...,np.newaxis]
                                
            else:
                fn_idx = np.random.randint(len(fn_list))
                img = np.array(Image.open(f"fn_img/img_{fn_list[fn_idx]}"))
                mask = np.array(Image.open(f"fn_mask/mask_{fn_list[fn_idx]}"))[...,np.newaxis]
        else:
            img = np.array(Image.open("train_object_512/"+img_name))
            mask = np.array(Image.open("masks_object_512/"+img_name))[...,np.newaxis]


        #apply augmentation
        if self.augmentation:
            sample = self.augmentation(image=img, mask=mask)
            img, mask = sample['image'], sample['mask']
        
        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=img, mask=mask)
            img, mask = sample['image'], sample['mask']

        # reshape for converting to tensor (모두 적용)
        sample = self.to_tensor(image=img, mask=mask)
        img, mask = sample['image'], sample['mask']
        
        return img/255, mask
    
    def __len__(self):
        return len(self.namelist)

def get_training_augmentation():
    transform = [
        albu.Transpose(p=0.5),   # 2배
        albu.RandomRotate90(3),  # 4배
        albu.ShiftScaleRotate(p = 1),
        albu.ColorJitter(p = 0.8),
        #albu.CLAHE(p = 0.5),
        #albu.RandomSizedCrop(min_max_height=(384, 512), height = 512, width = 512, p = 1),#(128,256)사이에서 랜덤하게 크롭하고 회전
        albu.GridDistortion(p = 0.2), # 2배
        albu.GridDropout(p=0.5, ratio = 0.3),
        ]
    return albu.Compose(transform)


def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')

def get_preprocessing(preprocessing_fn):
    _transform = [
        albu.Lambda(image = preprocessing_fn),
    ]
    return albu.Compose(_transform)

In [ ]:
namelist = []
with zipfile.ZipFile("/content/train_object_512/train_object.zip", 'r') as img_arch:
    namelist = img_arch.namelist().copy()
len(namelist)

In [ ]:
dataset = Kidney_Dataset(namelist, augmentation = get_training_augmentation(),  classes=1)


In [ ]:
img_id = 2

In [ ]:
img_id = 15 +img_id
plt.figure(figsize=(10,10))
img, mask = dataset[img_id]
plt.imshow(img.transpose([1,2,0])) # 원본 # permute는 축 변경
plt.imshow(mask[0,:,:], alpha=0.3) # 레이블
plt.show()

In [ ]:
import torch

class SAM(torch.optim.Optimizer):
    def __init__(self, params, base_optimizer, rho=0.05, **kwargs):
        assert rho >= 0.0, f"Invalid rho, should be non-negative: {rho}"

        defaults = dict(rho=rho, **kwargs)
        super(SAM, self).__init__(params, defaults)

        self.base_optimizer = base_optimizer(self.param_groups, **kwargs)
        self.param_groups = self.base_optimizer.param_groups

    @torch.no_grad()
    def first_step(self, zero_grad=False):
        grad_norm = self._grad_norm()
        for group in self.param_groups:
            scale = group["rho"] / (grad_norm + 1e-12)

            for p in group["params"]:
                if p.grad is None: continue
                e_w = p.grad * scale
                p.add_(e_w)  # climb to the local maximum "w + e(w)"
                self.state[p]["e_w"] = e_w

        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def second_step(self, zero_grad=False):
        for group in self.param_groups:
            for p in group["params"]:
                if p.grad is None: continue
                p.sub_(self.state[p]["e_w"])  # get back to "w" from "w + e(w)"

        self.base_optimizer.step()  # do the actual "sharpness-aware" update

        if zero_grad: self.zero_grad()

    def step(self, closure=None):
        raise NotImplementedError("SAM doesn't work like the other optimizers, you should first call `first_step` and the `second_step`; see the documentation for more info.")

    def _grad_norm(self):
        norm = torch.norm(
                    torch.stack([
                        p.grad.norm(p=2)
                        for group in self.param_groups for p in group["params"]
                        if p.grad is not None
                    ]),
                    p=2
               )
        return norm


def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

SEED = 0
seed_everything(SEED)


class DiceScore(smp.utils.base.Metric):
    __name__ = 'dice_score'

    def __init__(self, eps=1e-7, threshold=0.5, activation=None, ignore_channels=None, **kwargs):
        super().__init__(**kwargs)
        self.eps = eps
        self.threshold = threshold
        #self.activation = Activation(activation)
        self.ignore_channels = ignore_channels

    def forward(self, y_pr, y_gt):
        y_pr = smp.utils.functional._threshold(y_pr, threshold=self.threshold)
        y_pr, y_gt = smp.utils.functional._take_channels(y_pr, y_gt, ignore_channels=self.ignore_channels)
        tp = torch.sum(y_gt * y_pr)
        total = torch.sum(y_gt) + torch.sum(y_pr)
        score = (tp + self.eps) * 2 / (total + self.eps)
        return score


class DiceBCELoss(smp.utils.base.Loss):
    __name__ = 'dice_bce_loss'
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.diceloss = smp.utils.losses.DiceLoss()
        self.bceloss  = torch.nn.BCELoss()
        self.bceloss.__name__ = 'bce_loss'
    def forward(self, y_pr, y_gt):
        return 0.4 * self.diceloss.forward(y_pr, y_gt) + 0.6 * self.bceloss.forward(y_pr, y_gt)



class SamTrainEpoch(smp.utils.train.TrainEpoch):
    def __init__(self, model, loss, metrics, optimizer, device='cuda', verbose=True):
        super().__init__(
            model = model,
            loss = loss,
            metrics = metrics,
            optimizer = optimizer,
            device = device,
            verbose = verbose
        )

    def batch_update(self, x, y):
        self.optimizer.zero_grad()
        prediction = self.model.forward(x)

        #
        loss = self.loss(prediction, y)
        loss.backward()
        optimizer.first_step(zero_grad=True)

        self.loss(self.model.forward(x), y).backward()
        optimizer.second_step(zero_grad=True)

        return loss, prediction

    def run(self, dataloader):

        self.on_epoch_start()

        logs = {}
        loss_meter = smp.utils.meter.AverageValueMeter()
        metrics_meters = {metric.__name__: smp.utils.meter.AverageValueMeter() for metric in self.metrics}

        with tqdm(dataloader, desc=self.stage_name, file=sys.stdout, disable=not (self.verbose)) as iterator:
            
            for x, y in iterator:
                x, y = x.to(self.device), y.to(self.device)
                loss, y_pred = self.batch_update(x, y)

                # update loss logs
                loss_value = loss.cpu().detach().numpy()
                loss_meter.add(loss_value)
                loss_logs = {self.loss.__name__: loss_meter.mean}
                logs.update(loss_logs)

                # update metrics logs
                for metric_fn in self.metrics:
                    metric_value = metric_fn(y_pred, y).cpu().detach().numpy()
                    metrics_meters[metric_fn.__name__].add(metric_value)
                metrics_logs = {k: v.mean for k, v in metrics_meters.items()}
                logs.update(metrics_logs)

                new_logs = {}
                for key in logs.keys():
                    new_logs['train_'+key] = logs[key]
                wandb.log(new_logs)

                if self.verbose:
                    s = self._format_logs(logs)
                    iterator.set_postfix_str(s)
        return logs

In [ ]:
namelist = []
with zipfile.ZipFile("/content/train_object_512/train_object.zip", 'r') as img_arch:
    namelist = img_arch.namelist().copy()
len(namelist)


In [ ]:
# model save path
path = '/content/drive/MyDrive/kaggledrive/kidney/model/adaboost/'

In [ ]:
import wandb
!wandb login
# 04e6d67fe4bcbf0d12c5ec8957027496271c0245
wandb.init(project="hubmap_kidney", reinit = True)
train_name = 'adaboost'
wandb.run.name = train_name
wandb.run.save()

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: Currently logged in as: deagwon-bu (use `wandb login --relogin` to force relogin)


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


True

In [ ]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=True, random_state=0)

namelist = np.array(namelist)
train_names, test_names = train_test_split(namelist, test_size=0.1,random_state=1015)

target_fold_index = 1
for fold_index, (trn_idx, val_idx) in enumerate(kfold.split(namelist),1):
    print(fold_index)
    if fold_index == target_fold_index:
        train_names, test_names = namelist[trn_idx], namelist[val_idx]

        ENCODER = 'efficientnet-b7'
        ENCODER_WEIGHTS = 'imagenet'
        DEVICE = 'cuda'
        ACTIVATION = 'sigmoid'

        model = smp.Unet(
                    encoder_name=ENCODER, 
                    encoder_weights=ENCODER_WEIGHTS, 
                    in_channels = 3,
                    classes=1, 
                    activation = ACTIVATION
                )
        #loss = smp.utils.losses.DiceLoss()
        loss = DiceBCELoss()
        # 마찬가지로 mae_over_fscore의 pytorch metric version입니다.
        metrics = [ DiceScore(threshold=0.5)]
        #base_optimizer = Ralamb(model.parameters(), weight_decay=0.01)
        #optimizer = Lookahead(base_optimizer)
        base_optimizer = torch.optim.SGD
        optimizer = SAM(model.parameters(), base_optimizer, lr=0.1, momentum=0.9)

        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer, 10, 2,eta_min=1e-6) # 1e-6

        train_epoch = SamTrainEpoch(
            model, 
            loss=loss, 
            metrics=metrics, 
            optimizer=optimizer,
            device=DEVICE,
            verbose=True,
        )

        valid_epoch = smp.utils.train.ValidEpoch(
            model, 
            loss=loss, 
            metrics=metrics, 
            device=DEVICE,
            verbose=True,
        )

        BATCH_SIZE = 4
        train_dataset = Kidney_Dataset(train_names, augmentation = get_training_augmentation(),  classes=1)
        valid_dataset = Kidney_Dataset(test_names, classes=1)
        train_loader = DataLoader(train_dataset,  batch_size=BATCH_SIZE, shuffle=True, num_workers=4, persistent_workers = True)
        valid_loader = DataLoader(valid_dataset,  batch_size=20, shuffle=False, num_workers=2)

        # 한 폴드당 150 Epoch 수행
        NUM_EPOCH = 10
        min_score = np.Inf

        MODEL = train_name
        for i in range(0, NUM_EPOCH):
            print('\nEpoch: {}'.format(i))
            train_logs = train_epoch.run(train_loader)
            valid_logs = valid_epoch.run(valid_loader)

            new_logs = {}
            for key in valid_logs.keys():
                new_logs['valid_'+key] = valid_logs[key]
            wandb.log(new_logs)


            scheduler.step()
            valid_loss = valid_logs['dice_bce_loss']
            torch.save(model, f'{path}{fold_index}_{MODEL}_{loss.__name__}_{valid_loss:2.4f}_epoch_{i}.pth')
    
            print('Model saved!')
        target_fold_index += 1
        break

1


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b7-dcc49843.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b7-dcc49843.pth




Epoch: 0
valid: 100%|██████████| 65/65 [00:55<00:00,  1.17it/s, dice_bce_loss - 0.08579, dice_score - 0.9061]
Model saved!

Epoch: 1
valid: 100%|██████████| 65/65 [00:55<00:00,  1.18it/s, dice_bce_loss - 0.06823, dice_score - 0.9129]
Model saved!

Epoch: 2
valid: 100%|██████████| 65/65 [00:55<00:00,  1.18it/s, dice_bce_loss - 0.06369, dice_score - 0.9171]
Model saved!

Epoch: 3
valid: 100%|██████████| 65/65 [00:55<00:00,  1.18it/s, dice_bce_loss - 0.06075, dice_score - 0.9193]
Model saved!

Epoch: 4
valid: 100%|██████████| 65/65 [00:55<00:00,  1.18it/s, dice_bce_loss - 0.0611, dice_score - 0.9226]
Model saved!

Epoch: 5
valid: 100%|██████████| 65/65 [00:55<00:00,  1.18it/s, dice_bce_loss - 0.05626, dice_score - 0.9266]
Model saved!

Epoch: 6
valid: 100%|██████████| 65/65 [00:55<00:00,  1.18it/s, dice_bce_loss - 0.05474, dice_score - 0.9269]
Model saved!

Epoch: 7
valid: 100%|██████████| 65/65 [00:55<00:00,  1.18it/s, dice_bce_loss - 0.05423, dice_score - 0.9277]
Model saved!

Epoch: 

In [ ]:
checkpoint = { 
    'epoch': i,
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'lr_sched': scheduler
}
torch.save(checkpoint, path + 'checkpoint.pth')